In [31]:
import numpy as np

In [32]:
#Parameters
sequence = 30
delta = 1
K_max = 10 # Training = 1:NTD & Test = NTD:12
K = 3
epoch = 100
early = 10
lstm1 = 64
lstm2 = 64
random = 0 # on=1 off=0
KFold = 1 # on=1 off=0

In [33]:
import os

directory = r"/Users/stabatabaeim/Downloads/Data"  

# get a list of all .txt files in the directory
txt_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]

# print the number of .txt files found
print("Number of txt files:", len(txt_files))

# read the contents of each .txt file and store them in an array
Data = np.zeros((len(txt_files),50000,4))
for file_path in txt_files:
    for i in range(len(txt_files)):
        data = np.loadtxt(txt_files[i], dtype=object)
        data = [line.split(',') for line in data]

        # Convert the resulting list of lists to a NumPy array with the appropriate data types
        data = np.array(data, dtype=np.float32)
        #data = data.astype(np.int32)
        A = np.zeros((50000-data.shape[0], data.shape[1]))
        Data[i] = np.concatenate((data, A))


Number of txt files: 15


In [34]:
Data.shape

(15, 50000, 4)

In [35]:
# if KFold == 1:
#     Size = round(len(txt_files)*(1-TTP))
#     print("Size: ",Size)
#     data_zeros = np.zeros((Data.shape[0], Data.shape[1], Data.shape[2]),dtype='float') 
#     startpoint = (K-1)*Size
#     A = np.array(range(len(txt_files)))
#     for i in range(Size):
#         data_zeros[i] = Data[startpoint]
#         A = np.delete(A, Size*(K-1))
#         startpoint = startpoint + 1
#     for i in range(len(txt_files)-Size):
#         data_zeros[i+Size] = Data[A[i]]
#     Data = data_zeros

In [36]:
import pandas as pd

# Load the Excel file into a DataFrame
df = pd.read_excel(r"/Users/stabatabaeim/Downloads/Properties.xlsx", sheet_name='Sheet1')

# Select the values from row 3 and onwards
Properties = df.values[1:]

In [37]:
import numpy as np

# Assume the data array is already loaded into a variable named 'data'

# Find the indices where the values are "N" or "F"
idx_n = np.where(Properties == np.array(["N"], dtype=str))
idx_f = np.where(Properties == np.array(["F"], dtype=str))

# Replace "N" with "1"
Properties[idx_n] = 1

# Replace "F" with "2"
Properties[idx_f] = 2



In [38]:
Properties = np.array(Properties)
Properties = Properties.astype(float)

In [39]:
Properties.shape

(175, 19)

In [40]:
Properties[0]

array([  5.,   1.,   1.,   7.,   0.,   0., -75.,   1.,   2.,   7.,   0.,
         0.,  60.,   0.,   0.,   0.,   0.,   0.,   0.])

In [41]:
resulotion = 0.1
Dim = round((Properties[-1,0]-Properties[0,0])/resulotion)
Properties_res = np.zeros((Dim, Properties.shape[1]))
Data_res = np.zeros((len(txt_files), Dim, Data.shape[2]))
t0 = 5
tf = 620.8
t = t0
Data_inter = np.zeros(len(txt_files),dtype="int")
Properties_inter = 0
#for j in range(len(txt_files)):
for j in range(1):
    for i in range(Data[j].shape[0]):
        if Data[j,i,0] >= t0:
            Data_inter[j] = i - 1
            break
for i in range(Dim):
    Properties_res[i] = Properties[Properties_inter]
    Properties_res[i,0] = t
    if t >= Properties[Properties_inter+1,0]:
        Properties_inter += 1
    for j in range(len(txt_files)):
        if Data[j,Data_inter[j]+2,0]==0:
            continue
        Data_res[j,i] = Data[j, Data_inter[j]]
        Data_res[j,i,0] = t
        while True:
            if t >= Data[j, Data_inter[j] + 1,0]:
                Data_inter[j] += 1
            else:
                break
    
    t += resulotion

In [42]:
Data_res.shape

(15, 6158, 4)

In [43]:
Sit_change = []
Sit_change.append(-1)
array_without_angle_col = np.delete(Properties_res, [0,6,12,18], axis=1)
for i in range(Properties_res.shape[0]-1):
    if not np.array_equal(array_without_angle_col[i+1], array_without_angle_col[i]):
        Sit_change.append(i)
        
Sit_change.append(tf/resulotion)
Sit_change = np.array(Sit_change)
print(Sit_change.shape)
target_number = int(K/resulotion*tf/K_max)
nearest_index = np.abs(Sit_change - target_number).argmin()
Stop_K = int(Sit_change[nearest_index] + 1)
Stop_t = int(Properties_res[Stop_K,0]*100)/100

target_number = int((K-1)/resulotion*tf/K_max)
nearest_index = np.abs(Sit_change - target_number).argmin()
Start_K = int(Sit_change[nearest_index] + 1)
Start_t = int(Properties_res[Start_K,0]*100)/100
print(Start_t)
print(Stop_t)

(127,)
131.49
192.19


In [44]:
Stop_t-resulotion*sequence

189.19

In [45]:
np.array(Sit_change, dtype ='int')

array([  -1,   53,  101,  150,  200,  254,  300,  323,  355,  403,  457,
        505,  555,  608,  657,  713,  757,  811,  860,  910,  961, 1011,
       1061, 1114, 1171, 1211, 1264, 1314, 1368, 1416, 1467, 1517, 1566,
       1619, 1668, 1720, 1771, 1823, 1871, 1919, 1971, 2021, 2072, 2123,
       2174, 2221, 2274, 2326, 2377, 2429, 2466, 2529, 2578, 2628, 2681,
       2729, 2751, 2766, 2831, 2881, 2931, 2983, 3035, 3085, 3093, 3135,
       3182, 3231, 3287, 3335, 3388, 3440, 3493, 3540, 3590, 3640, 3689,
       3740, 3792, 3852, 3895, 3994, 4011, 4042, 4095, 4147, 4197, 4244,
       4291, 4344, 4395, 4445, 4501, 4550, 4600, 4650, 4698, 4750, 4802,
       4850, 4901, 4954, 5004, 5057, 5103, 5153, 5205, 5223, 5254, 5308,
       5359, 5403, 5456, 5506, 5525, 5557, 5610, 5662, 5712, 5760, 5810,
       5860, 5918, 5963, 6013, 6122, 6207])

In [46]:
# Number 1
XBminN1 = 120 #X Body min Near
XBmaxN1 = 160 #X Body max Near

XBminF1 = 150 #X Body min Far
XBmaxF1 = 170 #X Body max Far


# Number 2
XBminN2 = 160 #X Body min Near
XBmaxN2 = 195 #X Body max Near

XBminF2 = 170 #X Body min Far
XBmaxF2 = 190 #X Body max Far


# Number 3
XBminN3 = 195 #X Body min Near
XBmaxN3 = 230 #X Body max Near

XBminF3 = 190 #X Body min Far
XBmaxF3 = 220 #X Body max Far


In [47]:
Model_data = np.zeros((Data_res.shape[0]*Data_res.shape[1],Properties_res.shape[1]))
Model_label = np.zeros((Data_res.shape[0]*Data_res.shape[1]), dtype="O")
inter = 0
for z in range(Data_res.shape[0]):
    for j in range(Data_res.shape[1]):
        if Data_res[z,j,0] == 0:
            continue
        Model_data[inter] = Properties_res[j]
        Model_label[inter] = "No Data"

        P = 0 # number of person 1
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN1<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"

                        
            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF1<Data_res[z,j,2]<XBmaxF1: #Body
                        Model_label[inter] = "Number 1"

                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if 100<Data_res[z,j,2]<XBmaxN1: #Body
                        Model_label[inter] = "Number 1"


        P = 1 # number of person 2
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN2<Data_res[z,j,2]<XBmaxN2: #Body
                        Model_label[inter] = "Number 2"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF2<Data_res[z,j,2]<XBmaxF2: #Body
                        Model_label[inter] = "Number 2"

                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if Properties_res[j,6*P+6] <=0:  #if the person is leaving or entering from left side
                        if 100<Data_res[z,j,2]<180: #Body
                            Model_label[inter] = "Number 2"
                    elif Properties_res[j,6*P+6] >=0:  #if the person is leaving or entering from left side
                        if 180<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 2"
                            

        P = 2 # number of person 3
        if Properties_res[j,6*P+1] == 1: #if the person is present
            if Properties_res[j,6*P+2] == 1: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminN3<Data_res[z,j,2]<XBmaxN3: #Body
                        Model_label[inter] = "Number 3"
                            
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                    if XBminN3<Data_res[z,j,2]<250: #Body
                        Model_label[inter] = "Number 3"

            elif Properties_res[j,6*P+2] == 2: #if the person is Near or Far
                if Properties_res[j,6*P+3] != 7: #if the person is stationary or Entering or Leaving
                    if XBminF3<Data_res[z,j,2]<XBmaxF3: #Body
                        Model_label[inter] = "Number 3"
                        
                elif Properties_res[j,6*P+3] == 7:  #if the person is stationary or moving
                        if XBminF3<Data_res[z,j,2]<250: #Body
                            Model_label[inter] = "Number 3"
        inter +=1

In [48]:
Model_data = np.array(Model_data[0:inter])
Model_label = np.array(Model_label[0:inter])

In [49]:
Model_data.shape

(90257, 19)

In [50]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler = scaler.fit(Model_data)
Model_data_normal = scaler.transform(Model_data)  #scale the data

Model_data_normal = np.delete(Model_data_normal, 0, axis=1)

In [51]:
mean = scaler.mean_
std = scaler.scale_

In [52]:
print(mean[1:19])
print(std[1:19])

[  0.8220969    1.23059707   3.93203851   0.30711191   0.06636604
 -31.7745992    0.8045581    1.19866603   3.77597305   0.28457627
   0.16342223  -0.37476318   0.81636881   1.22212128   3.61347042
   0.24680634   0.07481968  31.26738092]
[ 0.38243115  0.7302248   2.76653838  0.4612962   0.24892085 17.26125689
  0.3965405   0.74167489  2.72222473  0.45121238  0.3941179  10.20974981
  0.38718313  0.73487808  2.63858312  0.43115307  0.29298619 17.33424739]


In [53]:
def Labell(String):
    if String == "Number 1":
        return 0
    elif String == "Number 2":
        return 1
    elif String == "Number 3":
        return 2

In [54]:
# Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0]-sequence+1,sequence,Model_data_normal.shape[1]),dtype='float')
# for i in range(Model_data_normal_sequence.shape[0]):
#     for j in range(sequence):
#         Model_data_normal_sequence[i,j] = Model_data_normal[i+j]
        
# Model_data_normal_sequence = np.delete(Model_data_normal_sequence, -1, axis = 0)

In [55]:
# Model_data_normal_sequence = np.zeros((Model_data_normal.shape[0],sequence,Model_data_normal.shape[1]),dtype='float')
# Model_label_sequence = np.zeros((Model_data_normal.shape[0],1),dtype='int')
# inter = 0
# Persons_inter = np.zeros(len(txt_files))
# inter2 = 0
# for i in range(Model_data_normal_sequence.shape[0]-sequence):
#     if Model_data[i, 0] == 5:
#         Persons_inter[inter2] = inter
#         inter2 += 1
#     if Model_label[i+sequence] != "No Data" and (Model_data[i+sequence, 0] - Model_data[i, 0]) > 0:
#         Model_label_sequence[inter] = Labell(Model_label[i+sequence])
#         for j in range(sequence):
#             Model_data_normal_sequence[inter,j] = Model_data_normal[i+j]
#         inter += 1
        
# Model_data_normal_sequence = np.array(Model_data_normal_sequence[0:inter])
# Model_label_sequence = np.array(Model_label_sequence[0:inter])

In [56]:
train_data_normal_sequence = np.zeros((Model_data_normal.shape[0],sequence,Model_data_normal.shape[1]),dtype='float')
train_label_sequence = np.zeros((Model_data_normal.shape[0],1),dtype='int')
test_data_normal_sequence = np.zeros((Model_data_normal.shape[0],sequence,Model_data_normal.shape[1]),dtype='float')
test_label_sequence = np.zeros((Model_data_normal.shape[0],1),dtype='int')
train_inter = 0
test_inter = 0
for i in range(train_data_normal_sequence.shape[0]-sequence):
    if Model_label[i+sequence] != "No Data" and (Model_data[i, 0] - Model_data[i-sequence, 0]) > 0:
        if Model_data[i, 0]>=Start_t and Model_data[i, 0]<Stop_t-resulotion*sequence:
            test_label_sequence[test_inter] = Labell(Model_label[i+sequence])
            for j in range(sequence):
                test_data_normal_sequence[test_inter,j] = Model_data_normal[i+j]
            test_inter += 1
                
        elif Model_data[i, 0]<Start_t-resulotion*sequence:
            train_label_sequence[train_inter] = Labell(Model_label[i+sequence])
            for j in range(sequence):
                train_data_normal_sequence[train_inter,j] = Model_data_normal[i+j]
            train_inter += 1
            
        elif Model_data[i, 0]>=Stop_t and Model_data[i, 0]<tf-resulotion*sequence:
            train_label_sequence[train_inter] = Labell(Model_label[i+sequence])
            for j in range(sequence):
                train_data_normal_sequence[train_inter,j] = Model_data_normal[i+j]
            train_inter += 1

train_data_normal_sequence = np.array(train_data_normal_sequence[0:train_inter])
train_label_sequence = np.array(train_label_sequence[0:train_inter])
test_data_normal_sequence = np.array(test_data_normal_sequence[0:test_inter])
test_label_sequence = np.array(test_label_sequence[0:test_inter])

In [57]:
test_inter

8246

In [58]:
train_inter/test_inter

8.859083191850594

In [59]:
train_data = np.array(train_data_normal_sequence)
train_label = np.array(train_label_sequence)
test_data = np.array(test_data_normal_sequence)
test_label = np.array(test_label_sequence)

In [60]:
print("train_data:",train_data.shape)
print("train_label:",train_label.shape)
print("test_data:",test_data.shape)
print("test_label:",test_label.shape)

train_data: (73052, 30, 18)
train_label: (73052, 1)
test_data: (8246, 30, 18)
test_label: (8246, 1)


In [61]:
from tensorflow.keras.utils import to_categorical

#X_train = train_data.reshape(train_data.shape[0],sequence, -1)
one_hot_encoded_labels_train = to_categorical(train_label,3)

#X_test = test_data.reshape(test_data.shape[0],sequence, -1)
one_hot_encoded_labels_test = to_categorical(test_label,3)

In [62]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

model = Sequential()
model.add(LSTM(lstm1, activation='tanh',input_shape=(train_data.shape[1],train_data.shape[2]), return_sequences= True))
model.add(LSTM(lstm2, activation='tanh',input_shape=(train_data.shape[1],train_data.shape[2]), return_sequences= False))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(one_hot_encoded_labels_train[0].shape[0], activation='softmax'))
model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 30, 64)         │        21,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,467 (212.76 KB)

 Trainable params: 54,467 (212.76 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = early, mode = 'max', restore_best_weights = True)
import keras
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

model_training_history = model.fit(x = train_data, y = one_hot_encoded_labels_train, epochs = 10 , batch_size = 20,
                                                    shuffle = True, validation_data=(test_data, one_hot_encoded_labels_test),
                                                    callbacks = [early_stopping_callback])



Epoch 1/10
3653/3653 ━━━━━━━━━━━━━━━━━━━━ 31s 8ms/step - accuracy: 0.6972 - loss: 0.6584 - val_accuracy: 0.6659 - val_loss: 0.8484
Epoch 2/10
3653/3653 ━━━━━━━━━━━━━━━━━━━━ 30s 8ms/step - accuracy: 0.7359 - loss: 0.5878 - val_accuracy: 0.6607 - val_loss: 0.8595
Epoch 3/10
 995/3653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.7412 - loss: 0.5735

KeyboardInterrupt: 

In [ ]:
y_pred_train = model.predict(train_data)
y_pred_test = model.predict(test_data)

In [ ]:
def accuracymodel1(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        a = a.argmax()
        if a==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [ ]:
accuracymodel1(y_pred_train,train_label)
accuracymodel1(y_pred_test,test_label)

In [ ]:
def accuracymodel2(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [ ]:
accuracymodel2(y_pred_train,train_label)
accuracymodel2(y_pred_test,test_label)

In [ ]:
def accuracymodel3(y_pred,train_label):
    SumCorrect = 0
    for i in range(y_pred.shape[0]):
        a = np.array(y_pred[i])
        b = np.array(y_pred[i])
        c = np.array(y_pred[i])
        a = a.argmax()
        b[a] = 0
        b = b.argmax()
        c[a] = 0
        c[b] = 0
        c = c.argmax()
        
        if a==train_label[i,0] or b==train_label[i,0] or c==train_label[i,0]:
            SumCorrect = SumCorrect + 1
    return SumCorrect/y_pred.shape[0]

In [ ]:
accuracymodel3(y_pred_train,train_label)
accuracymodel3(y_pred_test,test_label)

In [ ]:
# A = np.array([[accuracymodel1(y_pred_train,train_label), accuracymodel2(y_pred_train,train_label),accuracymodel3(y_pred_train,train_label)],
#              [accuracymodel1(y_pred_test,test_label), accuracymodel2(y_pred_test,test_label),accuracymodel3(y_pred_test,test_label)]])
# print(A)
# Name = "LSTMBody-K"+str(K)+"-NTD"+str(NTD)+"-sequence" +str(sequence)+"-delta"+str(delta)
# np.savetxt(r"F:\Courses\Paper\Mine\New\Process\Result\Real\Olds\Results"+"/"+Name+'.csv', A, delimiter=',')   # X is an array
# model.save(r"F:\Courses\Paper\Mine\New\Process\Result\Real\Olds\Results"+"/"+Name+'.h5')  # creates a HDF5 file 'my_model.h5'